In [3]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (20.2.4)
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=905919b50bce7ba95f4cde54983f70da22462fe934a2620e3ce545f03aa0feac
  Stored in directory: /home/ec2-user/.cache/pip/wheels/88/2a/d4/f2e9023989d4d4b3574f268657cb6cd23994665a038803f547
Successfully built smart-open


In [4]:
import boto3
from trp import Document, Page, BoundingBox
from PIL import Image, ImageDraw, ImageFont
from smart_open import open
from sagemaker.session import Session
import minecart
import pandas as pd
%matplotlib inline

In [5]:
import time 
import random 
import numpy as np

In [6]:
# initiate s3 bucket and corresponding data folder
# bucket = "ran-s3-systemic-risk"
# data_folder ="Input/X-17A-5/"

bucket = "ran-s3-systemic-risk"
data_folder ="FOCUS-OCR/SubsetTest/"

# balance_sheet_idx = 3  # the page of the report you care about

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))
subset = 1
test_key = np.random.choice(paths, subset)

In [7]:
# random sample of files from s3
test_key

array(['FOCUS-OCR/SubsetTest/BLACKROCK EXECUTION SERVICES04.pdf'],
      dtype='<U79')

In [13]:
# content modified from Amazon AWS Textract repository (refer to URL below)
# https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py

def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['TABLES']
    )
    
    # return response job ID for service
    return response["JobId"]

def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))

    return status

def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
   
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):

        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)

        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages


In [9]:
count = []

# S3 storage for files on AWS site   
for key in test_key:
    jobId = startJob(bucket, key)
    print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
                        

Started job with id: 73e8db2a6f3dfbc1ebd37f6f21b7c54a3f1518f1eea254e778c02459ed286d7f
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED

Document
Page
Line
03/31/04 WED 11:01 FAX 1 415 908 7126
Words
----------
[03/31/04][WED][11:01][FAX][1][415][908][7126]
Line
BGI FINANCE & TREAS
Words
----------
[BGI][FINANCE][&][TREAS]
Line
003
Words
----------
[003]
Line
14
Words
----------
[14]
Line
04017706
Words
----------
[04017706]
Line
OMB APPROVAL
Words
----------
[OMB][APPROVAL]
Line
SECURITIES A
Words
----------
[SECURITIES][A]
Line
ON
Words
----------
[ON]
Line
OMB Number:
Words
----------
[OMB][Number:]
Line
3235-0123
Words
----------
[3235-0123]
Line
Washington, D.C. 20549
Words
----------
[Washington,][D.C.][20549]
Line
Expires:
Words
----------
[Expires:]
Line
October 31, 2004
Words
----------
[October][31,][2004]
Line
Est

In [10]:
#  Print detected text from PDF files
# for resultPage in response:

#     if resultPage['JobStatus'] != 'FAILED':
#         for item in resultPage["Blocks"]:
#             if item["BlockType"] == "LINE":
#                 print(item['Text'])
#                 if item['Text'] in searchList:
#                     if item['Page'] > 10:
#                         print(key)
#                     count.append(item['Page'])

In [15]:
doc = Document(response)

In [85]:
def cleanNumeric(value:str) -> float:
    """
    This function converts a string to a numeric quantity, handles weird string format
    :param: value, string value with hidden numeric quanity  
    :return: floating point values
    
    Complexity -> O(n)
    
    e.g.
        In[0]: $ 19,225     ->   Out[0]: 19255
        In[0]: $ 19,225.76  ->   Out[0]: 19255.76
    """
    arr = ""
    for elm in value:
        try:
            if (elm == '.') or int(elm):
                arr += elm 
        except ValueError:
            pass
        
    return float(arr)

In [84]:
cleanNumeric("$ 19,226.5")

19226.5

In [95]:
arr = []

# iterate through document pages
for page in doc.pages:
    # itterate through page tables
    for table in page.tables:    
        # itterate through each row within the provided table
        for row in table.rows:
            # strip the text from the cell references
            n = len(row.cells)
            arr.append([row.cells[i].text.strip() for i in range(n)])

In [98]:
pd.DataFrame(np.array(arr), columns=['Figures', 'Value'])

,Figures,Value
0,Cash and cash equivalents,"$ 19,226"
1,"Marketable securities, at fair value",928
2,Deposit with clearing organization and deposit...,"1,017"
3,Commissions receivable,"2,747"
4,Receivable from affiliate,"1,075"
5,Prepaid expenses and other assets,633
6,"Furniture, equipment and leasehold improvement...",159
7,"Deferred tax asset, net (Note 6)","1,239"
8,Total assets,"$ 27,024"
9,Liabilities and shareholder's equity,


In [63]:
for page in doc.pages:
    # itterate through page tables
    for table in page.tables:
        for row in table.rows:
            print(row)

[Cash and cash equivalents ][$ 19,226 ]
[Marketable securities, at fair value ][928 ]
[Deposit with clearing organization and depository (Note 8) ][1,017 ]
[Commissions receivable ][2,747 ]
[Receivable from affiliate ][1,075 ]
[Prepaid expenses and other assets ][633 ]
[Furniture, equipment and leasehold improvements, at cost, less accumulated depreciation and amortization of $431 (Note 5) ][159 ]
[Deferred tax asset, net (Note 6) ][1,239 ]
[Total assets ][$ 27,024 ]
[Liabilities and shareholder's equity ][]
[Liabilities: ][]
[Transaction processing fees payable ][$ 866 ]
[Accounts payable and accrued liabilities ][105 ]
[Payable to affiliate ][29 ]
[Income tax payable ][1,555 ]
[Accrued employee compensation and benefits ][7,325 ]
[Other liabilities ][891 ]
[Total liabilities ][10,771 ]
[Shareholder's equity: ][]
[Common stock $.01 par value; 1,000 shares authorized; one ][]
[share issued and outstanding ][- ]
[Additional paid-in capital ][1,702 ]
[Unearned stock-based compensation (N